In [1]:
import os
os.environ["PYSPARK_PYTHON"] = "python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"

In [2]:
os.environ["PYSPARK_PYTHON"] = r"C:\Users\elomark\anaconda3\envs\elo-TF\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = r"C:\Users\elomark\anaconda3\envs\elo-TF\python.exe"

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
spark = SparkSession.builder.appName("Pyspark-Ml-Algorithms").getOrCreate()

In [5]:
dataFrame = spark.read.csv("Admission_Predict.csv", header=True)

In [6]:
type(dataFrame)

pyspark.sql.classic.dataframe.DataFrame

In [7]:
dataFrame.show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|
|      316|        104|                3|  3| 3.5|   8|       1|            0.72|
|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|      314|        103|                2|  2|   3|8.21|       0|            0.65|
|      330|        115|             NULL|4.5|   3|9.34|       1|             0.9|
|      321|        109|                3|  3|   4| 8.2|       1|            0.75|
|      308|        101|                2|  3|   4| 7.9|       0|            0.68|
|      302|        102|                1|  2| 1.5|   8|       0|             0.5|
|      323|     

In [8]:
dataFrame.printSchema()

root
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR : string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit : string (nullable = true)



In [9]:
dataFrame.columns

['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA',
 'Research',
 'Chance of Admit ']

In [10]:
from pyspark.sql.functions import col
for c in dataFrame.columns:
    print(col((c)))

Column<'GRE Score'>
Column<'TOEFL Score'>
Column<'University Rating'>
Column<'SOP'>
Column<'LOR '>
Column<'CGPA'>
Column<'Research'>
Column<'Chance of Admit '>


In [11]:
dataFrame.select(*(col(c) for c in dataFrame.columns)).show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|
|      316|        104|                3|  3| 3.5|   8|       1|            0.72|
|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|      314|        103|                2|  2|   3|8.21|       0|            0.65|
|      330|        115|             NULL|4.5|   3|9.34|       1|             0.9|
|      321|        109|                3|  3|   4| 8.2|       1|            0.75|
|      308|        101|                2|  3|   4| 7.9|       0|            0.68|
|      302|        102|                1|  2| 1.5|   8|       0|             0.5|
|      323|     

In [12]:
dataFrame.select(*(col(c).cast("float") for c in dataFrame.columns)).show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|    337.0|      118.0|              4.0|4.5| 4.5|9.65|     1.0|            0.92|
|    324.0|      107.0|              4.0|4.0| 4.5|8.87|     1.0|            0.76|
|    316.0|      104.0|              3.0|3.0| 3.5| 8.0|     1.0|            0.72|
|    322.0|      110.0|              3.0|3.5| 2.5|8.67|     1.0|             0.8|
|    314.0|      103.0|              2.0|2.0| 3.0|8.21|     0.0|            0.65|
|    330.0|      115.0|             NULL|4.5| 3.0|9.34|     1.0|             0.9|
|    321.0|      109.0|              3.0|3.0| 4.0| 8.2|     1.0|            0.75|
|    308.0|      101.0|              2.0|3.0| 4.0| 7.9|     0.0|            0.68|
|    302.0|      102.0|              1.0|2.0| 1.5| 8.0|     0.0|             0.5|
|    323.0|     

In [13]:
dataFrame.select(*(col(c).cast("float").alias(c) for c in dataFrame.columns)).show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|    337.0|      118.0|              4.0|4.5| 4.5|9.65|     1.0|            0.92|
|    324.0|      107.0|              4.0|4.0| 4.5|8.87|     1.0|            0.76|
|    316.0|      104.0|              3.0|3.0| 3.5| 8.0|     1.0|            0.72|
|    322.0|      110.0|              3.0|3.5| 2.5|8.67|     1.0|             0.8|
|    314.0|      103.0|              2.0|2.0| 3.0|8.21|     0.0|            0.65|
|    330.0|      115.0|             NULL|4.5| 3.0|9.34|     1.0|             0.9|
|    321.0|      109.0|              3.0|3.0| 4.0| 8.2|     1.0|            0.75|
|    308.0|      101.0|              2.0|3.0| 4.0| 7.9|     0.0|            0.68|
|    302.0|      102.0|              1.0|2.0| 1.5| 8.0|     0.0|             0.5|
|    323.0|     

In [14]:
new_dataFrame = dataFrame.select(*(col(c).cast("float") for c in dataFrame.columns))

In [15]:
new_dataFrame.printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR : float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit : float (nullable = true)



In [16]:
from pyspark.sql.functions import col, count, isnan, when

In [17]:
# Checking for null ir nan type value in our columns
new_dataFrame.select([count(when(col(c).isNull(), c)).alias(c) for c in new_dataFrame.columns]).show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|        0|          0|                1|  0|   0|   0|       0|               0|
+---------+-----------+-----------------+---+----+----+--------+----------------+



In [18]:
from pyspark.ml.feature import Imputer

In [19]:
imputer = Imputer(inputCols=["GRE Score","TOEFL Score","University Rating"],
                 outputCols=["GRE Score","TOEFL Score","University Rating"])

model = imputer.fit(new_dataFrame)
imputed_data = model.transform(new_dataFrame)

In [20]:
imputed_data.show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|    337.0|      118.0|              4.0|4.5| 4.5|9.65|     1.0|            0.92|
|    324.0|      107.0|              4.0|4.0| 4.5|8.87|     1.0|            0.76|
|    316.0|      104.0|              3.0|3.0| 3.5| 8.0|     1.0|            0.72|
|    322.0|      110.0|              3.0|3.5| 2.5|8.67|     1.0|             0.8|
|    314.0|      103.0|              2.0|2.0| 3.0|8.21|     0.0|            0.65|
|    330.0|      115.0|        3.0827067|4.5| 3.0|9.34|     1.0|             0.9|
|    321.0|      109.0|              3.0|3.0| 4.0| 8.2|     1.0|            0.75|
|    308.0|      101.0|              2.0|3.0| 4.0| 7.9|     0.0|            0.68|
|    302.0|      102.0|              1.0|2.0| 1.5| 8.0|     0.0|             0.5|
|    323.0|     

In [21]:
# Checking for null ir nan type value in our columns
imputed_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_dataFrame.columns]).show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|        0|          0|                0|  0|   0|   0|       0|               0|
+---------+-----------+-----------------+---+----+----+--------+----------------+



In [22]:
features = imputed_data.drop('Chance of Admit')

In [23]:
features.columns

['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA',
 'Research',
 'Chance of Admit ']

In [24]:
assembler = VectorAssembler(inputCols=features.columns, outputCol="features")

In [25]:
output = assembler.transform(imputed_data)

In [26]:
output.show()

+---------+-----------+-----------------+---+----+----+--------+----------------+--------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |            features|
+---------+-----------+-----------------+---+----+----+--------+----------------+--------------------+
|    337.0|      118.0|              4.0|4.5| 4.5|9.65|     1.0|            0.92|[337.0,118.0,4.0,...|
|    324.0|      107.0|              4.0|4.0| 4.5|8.87|     1.0|            0.76|[324.0,107.0,4.0,...|
|    316.0|      104.0|              3.0|3.0| 3.5| 8.0|     1.0|            0.72|[316.0,104.0,3.0,...|
|    322.0|      110.0|              3.0|3.5| 2.5|8.67|     1.0|             0.8|[322.0,110.0,3.0,...|
|    314.0|      103.0|              2.0|2.0| 3.0|8.21|     0.0|            0.65|[314.0,103.0,2.0,...|
|    330.0|      115.0|        3.0827067|4.5| 3.0|9.34|     1.0|             0.9|[330.0,115.0,3.08...|
|    321.0|      109.0|              3.0|3.0| 4.0| 8.2|     1.0|         

In [27]:
output.select("features").toPandas().values

array([[DenseVector([337.0, 118.0, 4.0, 4.5, 4.5, 9.65, 1.0, 0.92])],
       [DenseVector([324.0, 107.0, 4.0, 4.0, 4.5, 8.87, 1.0, 0.76])],
       [DenseVector([316.0, 104.0, 3.0, 3.0, 3.5, 8.0, 1.0, 0.72])],
       [DenseVector([322.0, 110.0, 3.0, 3.5, 2.5, 8.67, 1.0, 0.8])],
       [DenseVector([314.0, 103.0, 2.0, 2.0, 3.0, 8.21, 0.0, 0.65])],
       [DenseVector([330.0, 115.0, 3.0827, 4.5, 3.0, 9.34, 1.0, 0.9])],
       [DenseVector([321.0, 109.0, 3.0, 3.0, 4.0, 8.2, 1.0, 0.75])],
       [DenseVector([308.0, 101.0, 2.0, 3.0, 4.0, 7.9, 0.0, 0.68])],
       [DenseVector([302.0, 102.0, 1.0, 2.0, 1.5, 8.0, 0.0, 0.5])],
       [DenseVector([323.0, 108.0, 3.0, 3.5, 3.0, 8.6, 0.0, 0.45])],
       [DenseVector([325.0, 106.0, 3.0, 3.5, 4.0, 8.4, 1.0, 0.52])],
       [DenseVector([327.0, 111.0, 4.0, 4.0, 4.5, 9.0, 1.0, 0.84])],
       [DenseVector([328.0, 112.0, 4.0, 4.0, 4.5, 9.1, 1.0, 0.78])],
       [DenseVector([307.0, 109.0, 3.0, 4.0, 3.0, 8.0, 1.0, 0.62])],
       [DenseVector([311.0, 1

In [28]:
# Rename column by trimming the trailing space
output = output.withColumnRenamed("Chance of Admit ", "Chance of Admit")

In [29]:
data = output.select(['features','Chance of Admit'])

In [30]:
train_df,test_df = data.randomSplit([0.7,0.3])

In [31]:
train_df.show()
test_df.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[290.0,100.0,1.0,...|           0.47|
|[290.0,104.0,4.0,...|           0.45|
|[293.0,97.0,2.0,2...|           0.64|
|[294.0,93.0,1.0,1...|           0.46|
|[294.0,95.0,1.0,1...|           0.49|
|[295.0,93.0,1.0,2...|           0.46|
|[295.0,96.0,2.0,1...|           0.47|
|[295.0,99.0,2.0,2...|           0.57|
|[295.0,101.0,2.0,...|           0.69|
|[296.0,95.0,2.0,3...|           0.44|
|[296.0,97.0,2.0,1...|           0.49|
|[296.0,99.0,2.0,3...|           0.47|
|[296.0,101.0,1.0,...|            0.6|
|[297.0,98.0,2.0,2...|           0.59|
|[298.0,92.0,1.0,2...|           0.51|
|[298.0,98.0,2.0,1...|           0.44|
|[298.0,99.0,1.0,1...|           0.53|
|[298.0,99.0,2.0,4...|           0.46|
|[298.0,101.0,2.0,...|           0.54|
|[298.0,105.0,3.0,...|           0.69|
+--------------------+---------------+
only showing top 20 rows
+--------------------+---------------+


###
### Linear Regressor

In [32]:
lin_reg = LinearRegression(featuresCol = 'features', labelCol = 'Chance of Admit')
linear_model = lin_reg.fit(train_df)

In [33]:
print("Coefficient : ", linear_model.coefficients)
print("Intercept : ", linear_model.intercept)

Coefficient :  [7.905507283742658e-18,1.4076139286048785e-17,5.499844839031365e-17,-1.8056499317487245e-17,1.2721218366130576e-16,8.617706226424688e-16,1.1805296340703974e-16,0.9999999999999936]
Intercept :  -7.378314133176692e-15


In [34]:
trainSummary = linear_model.summary
print("RMSE : ", trainSummary.rootMeanSquaredError)
print("r2 Score : ", trainSummary.r2)

RMSE :  3.8455036473014413e-16
r2 Score :  1.0


In [35]:
# Predictions
predictions = linear_model.transform(test_df)
predictions.select('prediction','Chance of Admit','features').show()

+-------------------+---------------+--------------------+
|         prediction|Chance of Admit|            features|
+-------------------+---------------+--------------------+
| 0.6100000143051145|           0.61|[296.0,99.0,2.0,2...|
|  0.430000007152558|           0.43|[297.0,96.0,2.0,2...|
| 0.3400000035762796|           0.34|[297.0,96.0,2.0,2...|
| 0.5199999809265139|           0.52|[297.0,100.0,1.0,...|
| 0.3400000035762802|           0.34|[298.0,98.0,2.0,4...|
|0.41999998688697826|           0.42|[299.0,94.0,1.0,1...|
| 0.5899999737739561|           0.59|[299.0,100.0,1.0,...|
| 0.5600000023841867|           0.56|[299.0,102.0,3.0,...|
| 0.5600000023841858|           0.56|[300.0,102.0,2.0,...|
| 0.6399999856948849|           0.64|[301.0,99.0,2.0,3...|
| 0.6800000071525569|           0.68|[301.0,104.0,2.0,...|
| 0.6800000071525573|           0.68|[301.0,104.0,3.0,...|
| 0.6200000047683719|           0.62|[301.0,107.0,3.0,...|
|  0.569999992847442|           0.57|[302.0,99.0,1.0,2..

In [36]:
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol='prediction',labelCol='Chance of Admit',metricName='r2')
print("R Square (R2) on the test data =",pred_evaluator.evaluate(predictions))

R Square (R2) on the test data = 1.0


###
### Random Forest Regressor

In [37]:
random_forest_reg = RandomForestRegressor(featuresCol='features',labelCol='Chance of Admit')

In [38]:
# Train Model. This also runs the indexer
model = random_forest_reg.fit(train_df)

In [39]:
# Make Predictions
predictions = model.transform(test_df)

In [40]:
predictions.show()

+--------------------+---------------+-------------------+
|            features|Chance of Admit|         prediction|
+--------------------+---------------+-------------------+
|[296.0,99.0,2.0,2...|           0.61| 0.6005639699074943|
|[297.0,96.0,2.0,2...|           0.43|  0.486790412696933|
|[297.0,96.0,2.0,2...|           0.34|0.46337689387647396|
|[297.0,100.0,1.0,...|           0.52| 0.5373538509598325|
|[298.0,98.0,2.0,4...|           0.34|  0.495348864780944|
|[299.0,94.0,1.0,1...|           0.42| 0.4631238544717752|
|[299.0,100.0,1.0,...|           0.59| 0.5798857984530289|
|[299.0,102.0,3.0,...|           0.56| 0.5763760497725106|
|[300.0,102.0,2.0,...|           0.56| 0.5315754679615615|
|[301.0,99.0,2.0,3...|           0.64| 0.6320886517349983|
|[301.0,104.0,2.0,...|           0.68| 0.6490462291685637|
|[301.0,104.0,3.0,...|           0.68| 0.6675965663143504|
|[301.0,107.0,3.0,...|           0.62| 0.6446628862123803|
|[302.0,99.0,1.0,2...|           0.57| 0.522965739403516

In [41]:
evaluator = RegressionEvaluator(predictionCol='prediction',labelCol='Chance of Admit',metricName='rmse')
print("Root Mean Square Error(RMSE) on the test data =",evaluator.evaluate(predictions))

Root Mean Square Error(RMSE) on the test data = 0.03297089534128378


In [42]:
evaluator = RegressionEvaluator(predictionCol='prediction',labelCol='Chance of Admit',metricName='r2')
print("R Square (R2) on the test data =",evaluator.evaluate(predictions))

R Square (R2) on the test data = 0.9454751932071624
